In [1]:
import numpy as np
import pandas as pd
from googleapiclient.discovery import build
from datetime import timedelta

# Display options:
pd.set_option("display.width", 1200)
pd.set_option("display.max_columns", 300)
pd.set_option("display.max_rows", 300)

In [4]:
# Youtube Data API:
yt = build(
    serviceName = "youtube",
    version = "v3",
    developerKey = "" # my key
)

In [2]:
### Channels sample

# * the API only returns 20.000 videos by channel
# Ids from (content = "https://www.youtube.com/channel/...) and titles:
channels = {
    "UCYO_jab_esuFRV4b17AJtAw": "3Blue1Brown",
    "UCNKcMBYP_-18FLgk4BYGtfw": "AWE me",
    "UCiDJtJKMICpb9B1qf7qjEOA": "Adam Savage’s Tested",
    "UC32z4mtyiq02Ge-XWy78ibw": "Aero Por Trás da Aviação",
    "UCaRMivfyupj3ucUyJbZbCNg": "Anglo-Link",
    "UCwg6_F2hDHYrqbNSGjmar4w": "Animalogic",
    "UCi5iiEyLwSLvlqnMi02u5gQ": "Ask a Mortician",
    "UC-9b7aDP6ZN0coj9-xFnrtw": "Astrum",
    "UCSTlOTcyUmzvhQi6F8lFi5w": "Atila Iamarino",
    "UCLXl1V6n82Dyg1VhVgSL0nw": "Aviões e Músicas com Lito Sousa",
    "UCGlmclFVnJCFQ_VO4kTgTig": "Azusa Barbie",
    "UCCj956IF62FbT7Gouszaj9w": "BBC",
    "UCxcnsr1R5Ge_fbTu5ajt8DQ": "Bob Ross",
    "UCQRPDZMSwXFEDS67uc7kIdg": "Buenas Ideias",
    "UCcyq283he07B7_KUX07mmtA": "Business Insider",
    "UCvJJ_dzjViJCoLf5uKUTwoA": "CNBC",
    "UCWq1xltHB2fDe6YkYoOrryg": "Canal do Schwarza",
    "UCwnKziETDbHJtx78nIkfYug": "CaspianReport",
    "UCR1-7g_y2YcYlh1W9y_1LUg": "Chaves Estranho",
    "UC04KsGq3npibMCE9Td3mVDg": "Cheddar",
    "UCpVm7bg6pXKo1Pr6k5kxG9A": "Classic Mr Bean",
    "UCvmijL-eepDVHYSJHDY3d6w": "Cole and Marmalade",
    "UC9-y-6csu5WGm29I7JiwpnA": "Computerphile",
    "UCyAMV63OY0DRbKk6gGv6wJg": "Cradle Of Filth",
    "UC726J5A0LLFRxQ0SZqr2mYQ": "Curious Droid",
    "UCFuIUoyHB12qpYa8Jpxoxow": "Código Fonte TV",
    "UCd-swDW3HCs4LTZhq8Cf7Fg": "DW Euromaxx",
    "UCa37IMrH8BGS_pO6CKbTL-A": "Dinosaurs",
    "UC_5niPa-d35gg88HaS7RrIw": "Disney",
    "UC1xwwLwm6WSMbUn_Tp597hQ": "Disney Parks",
    "UCVRrGAcUc7cblUzOhI1KfFg": "Earthling Ed",
    "UCIBaDdAbGlFDeS33shmlD0A": "European Space Agency, ESA",
    "UC6uKrU_WqJ1R2HMTY3LIx5Q": "Everyday Astronaut",
    "UCZ6JHFBaDUJ9wfo41HSII_w": "Fabio Chaves",
    "UC6zbvGt_jJVgw7-MWkVUC5A": "Fala Vegan",
    "UCxGiUjep8KiihvHldRousPA": "Fancy Fairy Wings & Things",
    "UCeSRjhfeeqIgr--AcP9qhyg": "Guinness World Records",
    "UCLAcytNR3gdw44yzoSiKluA": "Hello Korea",
    "UC26YLK0OEbLB3TCYxGh8xVQ": "Huygens Optics",
    "UCcMDMoNu66_1Hwi5-MeiQgw": "Hydraulic Press Channel",
    "UCkAbeyJZqp6JFYfKy-98NjA": "Integrando Conhecimento",
    "UCJ8bjM5yQSkOP-_99n1zKkw": "Intensivo Pedagógico",
    "UCZ9l_6_f0PWRYXN5Y7Lcl2A": "Jacques Slade",
    "UCbbQalJ4OaC0oQ0AqRaOJ9g": "Jay Foreman",
    "UCkP2CvRubyU0MTZv_Qo712g": "Jessica in the Kitchen - Vegan Recipes",
    "UC-2YHgc363EdcusLIBbgxzg": "Joe Scott",
    "UCBbnbBWJtwsf0jLGUwX5Q3g": "Journey to the Microcosmos",
    "UCLQWhXC5aOZ_tQNloBg40BA": "Kirsten & Joerg",
    "UCuCuEKq1xuRA0dFQj1qg9-Q": "Knowledgia",
    "UCsXVk37bltHxD1rDPwtNM8Q": "Kurzgesagt – In a Nutshell",
    "UCNqNkZ7kKfqimqHkgbWMNYA": "Launch Pad Astronomy",
    "UCoNTMWgGuXtGPLv9UeJZwBw": "Living Big In A Tiny House",
    "UCt2WVZXVrHaFgZ432s33fLw": "MAKE UP FOR EVER",
    "UCBlXovStrlQkVA2xJEROUNg": "Marcel Vos",
    "UCr22xikWUK2yUW4YxOKXclQ": "MathTheBeautiful",
    "UC6nZXhOz_gHr4mlb5UNl9ug": "Matt Jordan",
    "UCQwFuQLnLocj5F7ZcmcuWYQ": "MetaBallStudios",
    "UCk5BcU1rOy6hepflk7_q_Pw": "Meteoro Brasil",
    "UCqONzeACDBaF6FfKjh7ndAQ": "Microsoft Flight Simulator",
    "UChHS4NI6U4XgCuYgsrygVCA": "Mintfaery",
    "UC1ZBQ-F-yktYD4m5AzM6pww": "Mustard",
    "UCLA_DiR1FfKNvjuUpBHmylQ": "NASA",
    "UCPZvqkyXoYDlzBClxpxVIzg": "Naomi Farr",
    "UCggHsHce2n3vvbJf_8YKrMA": "Nerdforge",
    "UCWOA1ZGywLbqmigxE4Qlvuw": "Netflix",
    "UCD57tGnYPW1twCohgIVkJpw": "Nox et Lux",
    "UCoxcjq-8xIDTYp3uz647V5A": "Numberphile",
    "UCroDJPcFCf6DBmHns6Xeb8g": "Nyma Tang",
    "UCQ3JxE-NOyZaJ3m3qIkZbhA": "O Pimentinha",
    "UCtb2f3j_sB4Z4Ik2YJtbzsg": "OneikaTraveller",
    "UCwK-svlNEASA7g_smu1_d8g": "OnlyConnect Fan",
    "UCpLQXR116cLVUa1LRY8KS4w": "OwlKitty",
    "UCzR-rom72PHN9Zg7RML9EbA": "PBS Eons",
    "UC7_gcs09iThXybpVgjHZ_7g": "PBS Space Time",
    "UCddYq41_tZ1FnLlguLT6-Ow": "Parafernalha",
    "UCOVkkVaUP0Xop6IIjyk86QA": "Perception",
    "UCtESv1e7ntJaLJYKIO1FoYw": "Periodic Videos",
    "UC7DdEm33SyaTDtWYGO2CwdA": "Physics Girl",
    "UCQwHYU3ZbSLG6LOVwtvuozg": "Planeta Aves",
    "UCEWHPFNilsT0IfQfutVzsag": "Porta dos Fundos",
    "UClo-U5gvXPRy-4VqWEkkRDg": "Primitive Building",
    "UCKKJpBveT8vWVNfLQ-MvZMg": "Prof. André Azevedo da Fonseca",
    "UCXOzDw_X92bNqSCqw-NojiA": "Professor Julio Borbo",
    "UCwSxSJqGpSRpEsq5-YUbM8g": "Professor Noslen",
    "UCNzul4dnciIlDg8BAcn5-cQ": "Prowalk Tours",
    "UCvn_XCl_mgQmt3sD753zdJA": "Rachel's English",
    "UCR1IuLEqb6UEA_zQ81kwXfg": "Real Engineering",
    "UCP5tjEmvPItGyLhmjdwP7Ww": "RealLifeLore",
    "UCFJxE0l3cVYU4kHzi4qVEkw": "Rebecca Watson",
    "UCliwOcLaEfqkGJ7xq2ipq4w": "Reverse Engineering",
    "UC9Ep0Y4T5rvUuIfjKN1wqTw": "Riddle",
    "UCovtFObhY9NypXcyHxAS7-Q": "Roberto Blake",
    "UCZYTClx2T1of7BRZ86-8fow": "SciShow",
    "UCxzC4EngIsMrPmbm6Nxvb-A": "Scott Manley",
    "UC3KEoMzNz8eYnwBC34RaKCQ": "Simone Giertz",
    "UCT3wwJBh8fwePiNC63NlzCA": "Simple Flying",
    "UC6107grRI4m0o2-emgoDnAA": "SmarterEveryDay",
    "UC_Fk7hHbl7vv_7K8tYqJd5A": "SpaceToday",
    "UCtI0Hodo5o5dUb67FeUjDeA": "SpaceX",
    "UCZ9jWH_8tJ-Nmaj8dSQdEYA": "Stefan Milo",
    "UCpQLC-evmUAon9BBpcW4kYg": "Steve Cutts",
    "UC6u6uY4VbvuNtU0BU7F9olw": "Studson Studio",
    "UCRXRbi80k0_vcIfgpOSerTg": "Subject Zero Science",
    "UCAuUUnT6oDeKwE6v1NGQxug": "TED",
    "UCZdJE8KpuFm6NRafHTEIC-g": "Tempero Drag",
    "UC6n8I1UDTKP1IWjQMg6_TwA": "The B1M",
    "UCvpQ-l09fCVxJd3urZbxzHg": "The British Museum",
    "UCsQXQMAZTygXddj_I4Ud9xA": "The Green Witch",
    "UCUK0HBIBWgM2c4vsPhkYY4w": "The Slow Mo Guys",
    "UC2LVhJH_9cT2XKp0VAfsKOQ": "The Tim Traveller",
    "UCBa659QWEk1AI4Tg--mrJ2A": "Tom Scott",
    "UClVbhSLxwws-KSsPKz135bw": "TopMovieClips",
    "UC29ju8bIPH5as8OGnQzwJyA": "Traversy Media",
    "UCgtq_tKnFFQyJ-7j0yr9utg": "Tyta Montrase",
    "UCSxFcMDGmwgDlSP9qmYVx1w": "TÁ NA MESA VEGG",
    "UCVEVuanoMK9tGclfWLghaKw": "Tá Querida",
    "UCTlLFaGX4q2otO0ZVxLW0VQ": "Uma história a mais",
    "UC9fSZHEh6XsRpX-xJc6lT3A": "UrAvgConsumer",
    "UCp3iXxis9n_E_GfbE-_ksFw": "VeganBlackMetalChef",
    "UCHnyfMqiRRG1u-2MsSQLbXA": "Veritasium",
    "UCYulrOk4EbqL833oETw0bWQ": "Wanderlust Travel Videos",
    "UCmVa-cbCpkd5Cd9Fr_4tCWg": "Your Dinosaurs Are Wrong",
    "UCXvA7wLxIJDa1SWYWl7e-oQ": "arewethereyet",
    "UCS4iXNXsvlOHm7TuXLltreg": "chilli music",
    "UC9Ntx-EF3LzKY1nQ5rTUP2g": "cyriak",
    "UCVpankR4HtoAVtYnFDUieYA": "zefrank1"
 }

In [6]:
###### Videos data

# Loop in the channels:
df_videos = pd.DataFrame(
    {
        "channel_title": pd.Series([], dtype = "str"),
        "channel_id": pd.Series([], dtype = "str"),
        "video_title": pd.Series([], dtype = "str"),
        "video_id": pd.Series([], dtype = "str"),
        "video_upload_date": pd.Series([], dtype = "str"),
        "views": pd.Series([], dtype = "int"),
        "likes": pd.Series([], dtype = "int"),
        "dislikes": pd.Series([], dtype = "int"),
        "comments": pd.Series([], dtype = "int")
    }
)
for chnl_ind, chnl_key in enumerate(channels):

    # Choose the channel:
    channel_id = chnl_key
    channel_title = channels[channel_id]

    print(channel_title)

    ### Extract the data from all the videos of the channel

    # Content details:
    content = yt.channels().list(
        id = channel_id,
        part = "contentDetails"
    ).execute()

    # Upload Id:
    upload_id = content["items"][0]["contentDetails"]["relatedPlaylists"]["uploads"]

    # All the channel's videos:
    all_videos = []
    next_pg_token = None
    res = yt.playlistItems().list(
        playlistId = upload_id,
        maxResults = 50,
        part = "snippet",
        pageToken = next_pg_token
    ).execute()
    while True:
        res = yt.playlistItems().list(
            playlistId = upload_id,
            maxResults = 50,
            part = "snippet",
            pageToken = next_pg_token
        ).execute()
        all_videos += res["items"]
        next_pg_token = res.get("nextPageToken")
        if next_pg_token is None:
            break

    # Videos statistics:
    videos_ids = list(map(lambda x: x["snippet"]["resourceId"]["videoId"], all_videos))
    stats = []
    for i in range(0, len(videos_ids), 40):
        res = (yt).videos().list(
            id = ",".join(videos_ids[i:i + 40]),
            part = "statistics"
        ).execute()
        stats += res["items"]

    # Put it in a dataframe:
    video_id, views, likes, dislikes, comments = [], [], [], [], []
    for i in range(len(stats)):
        video_id += [stats[i]["id"]]
        stats_i = stats[i]["statistics"]
        views += [stats_i[k] if "viewCount" in stats_i.keys() else np.nan for k in ["viewCount"]]
        likes += [stats_i[k] if "likeCount" in stats_i.keys() else np.nan for k in ["likeCount"]]
        dislikes += [stats_i[k] if "dislikeCount" in stats_i.keys() else np.nan for k in ["dislikeCount"]]
        comments += [stats_i[k] if "commentCount" in stats_i.keys() else np.nan for k in ["commentCount"]]
    df_channel_videos = pd.DataFrame(
        {
            "channel_title": channel_title,
            "channel_id": channel_id,
            "video_title": list(map(lambda x: x["snippet"]["title"], all_videos)),
            "video_id": video_id,
            "video_upload_date": list(map(lambda x: x["snippet"]["publishedAt"], all_videos)),
            "views": views,
            "likes": likes,
            "dislikes": dislikes,
            "comments": comments        
        }
    )
    df_videos = pd.concat([df_videos, df_channel_videos])

3Blue1Brown
AWE me
Adam Savage’s Tested
Aero Por Trás da Aviação
Anglo-Link
Animalogic
Ask a Mortician
Astrum
Atila Iamarino
Aviões e Músicas com Lito Sousa
Azusa Barbie
BBC
Bob Ross
Buenas Ideias
Business Insider
CNBC
Canal do Schwarza
CaspianReport
Chaves Estranho
Cheddar
Classic Mr Bean
Cole and Marmalade
Computerphile
Cradle Of Filth
Curious Droid
Código Fonte TV
DW Euromaxx
Dinosaurs
Disney
Disney Parks
Earthling Ed
European Space Agency, ESA
Everyday Astronaut
Fabio Chaves
Fala Vegan
Fancy Fairy Wings & Things
Guinness World Records
Hello Korea
Huygens Optics
Hydraulic Press Channel
Integrando Conhecimento
Intensivo Pedagógico
Jacques Slade
Jay Foreman
Jessica in the Kitchen - Vegan Recipes
Joe Scott
Journey to the Microcosmos
Kirsten & Joerg
Knowledgia
Kurzgesagt – In a Nutshell
Launch Pad Astronomy
Living Big In A Tiny House
MAKE UP FOR EVER
Marcel Vos
MathTheBeautiful
Matt Jordan
MetaBallStudios
Meteoro Brasil
Microsoft Flight Simulator
Mintfaery
Mustard
NASA
Naomi Farr
Nerdfo

In [7]:
df_videos

,channel_title,channel_id,video_title,video_id,video_upload_date,views,likes,dislikes,comments
0,3Blue1Brown,UCYO_jab_esuFRV4b17AJtAw,A few of the best math explainers from this su...,F3Qixy-r_rQ,2021-10-23T18:11:23Z,421163,27170,95,718
1,3Blue1Brown,UCYO_jab_esuFRV4b17AJtAw,How a Mandelbrot set arises from Newton’s work,LqbZpur38nw,2021-10-15T16:41:50Z,568689,26102,146,1216
2,3Blue1Brown,UCYO_jab_esuFRV4b17AJtAw,Newton's Fractal (which Newton knew nothing ab...,-RdOwhmqP5s,2021-10-07T02:19:39Z,1195545,61723,284,2810
3,3Blue1Brown,UCYO_jab_esuFRV4b17AJtAw,The Summer of Math Exposition,ojjzXyQCzso,2021-07-16T15:37:16Z,610550,29300,215,1719
4,3Blue1Brown,UCYO_jab_esuFRV4b17AJtAw,A quick trick for computing eigenvalues | Chap...,e50Bj7jn9IQ,2021-05-07T19:01:16Z,421914,17032,145,1146
...,...,...,...,...,...,...,...,...,...
191,zefrank1,UCVpankR4HtoAVtYnFDUieYA,HardTimes :: Hole,LPfcGXMpKds,2009-06-06T22:16:56Z,90817,1793,23,75
192,zefrank1,UCVpankR4HtoAVtYnFDUieYA,HardTimes :: Affordable Button,h3lvSflNixI,2009-05-30T00:13:46Z,103164,1566,47,18
193,zefrank1,UCVpankR4HtoAVtYnFDUieYA,HardTimes :: Optical Illusion,8mS5RK0Yo6w,2009-05-23T22:36:58Z,1072190,9848,1201,NaN
194,zefrank1,UCVpankR4HtoAVtYnFDUieYA,HardTimes :: Outsource,zxIbQ6ZG4OI,2009-05-17T00:50:31Z,78211,1620,20,87


In [25]:
# Remove the videos with NaN in any variable:
df_videos = df_videos.dropna()
df_videos = df_videos.reset_index(drop = True)

In [29]:
df_videos = df_videos.astype({
    "views": "int64",
    "likes": "int64",
    "dislikes": "int64",
    "comments": "int64"
})

In [36]:
df_videos.dtypes

channel_title                     object
channel_id                        object
video_title                       object
video_id                          object
video_upload_date    datetime64[ns, UTC]
views                              int64
likes                              int64
dislikes                           int64
comments                           int64
age_days                         float64
dtype: object

In [ ]:
###### Data wrangling

In [37]:
### Age in days (until the last upload date)

# Change to datetime:
df_videos["video_upload_date"] = pd.to_datetime(df_videos["video_upload_date"])

# Filter the wrong upload dates (coming from the API as being the request date) considering a margin of 2 days:
last_date = max(df_videos["video_upload_date"])
age = []
for i in range(df_videos.shape[0]):
    date_i = df_videos["video_upload_date"].iloc[i]
    diff_days = last_date - date_i
    if diff_days < timedelta(days = 2):
        age += [np.nan]
    else:
        age += [diff_days.days]
df_videos["age_days"] = age

df_videos = df_videos.dropna()
df_videos = df_videos.reset_index(drop = True)

In [38]:
df_videos["age_days"].describe()

count    117555.000000
mean       1857.664421
std        1219.396580
min           2.000000
25%         925.000000
50%        1684.000000
75%        2581.000000
max        5732.000000
Name: age_days, dtype: float64

In [39]:
df_videos.sort_values("age_days").apply(lambda x: pd.concat([x.head(1), x.tail(1)]))

,channel_title,channel_id,video_title,video_id,video_upload_date,views,likes,dislikes,comments,age_days
6860,Atila Iamarino,UCSTlOTcyUmzvhQi6F8lFi5w,Live 21/10/21 - O que vem pela frente na pandemia,r0t-IXDjI4k,2021-10-21 00:50:41+00:00,139209,20930,192,724,2.0
71027,Jacques Slade,UCZ9l_6_f0PWRYXN5Y7Lcl2A,Get Your Hands Up,FXLXBfmlaOk,2006-02-12 20:31:24+00:00,28227,206,9,108,5732.0


In [42]:
### Likes-dislikes ratio

df_videos["likes_dislikes_ratio"] = df_videos["likes"]/df_videos["dislikes"]
df_videos["likes_dislikes_ratio"] = df_videos["likes_dislikes_ratio"].replace([np.inf, -np.inf], np.nan)
df_videos = df_videos.dropna()
df_videos = df_videos.reset_index(drop = True)

In [46]:
df_videos["likes_dislikes_ratio"].describe()

count    109850.000000
mean         56.085996
std          78.512051
min           0.000000
25%          12.793772
50%          32.076923
75%          68.630233
max        2635.000000
Name: likes_dislikes_ratio, dtype: float64

In [47]:
df_videos.sort_values("likes_dislikes_ratio").apply(lambda x: pd.concat([x.head(1), x.tail(1)]))

,channel_title,channel_id,video_title,video_id,video_upload_date,views,likes,dislikes,comments,age_days,likes_dislikes_ratio
34642,CNBC,UCvJJ_dzjViJCoLf5uKUTwoA,Financial Crises Tend To Presage Far-Right Vot...,E91yDVtlzpg,2016-11-22 21:44:33+00:00,246,0,2,1,1795.0,0.0
107094,TÁ NA MESA VEGG,UCSxFcMDGmwgDlSP9qmYVx1w,"COMO ARMAZENAR FOLHAS, ORGANIZAR FEIRA + SAL T...",GGqRCQmtx-U,2021-05-21 01:16:50+00:00,15346,2635,1,261,155.0,2635.0


In [51]:
### Comments-views ratio

df_videos["comments_views_ratio"] = df_videos["comments"]/df_videos["views"]
df_videos["comments_views_ratio"] = df_videos["comments_views_ratio"].replace([np.inf, -np.inf], np.nan)

# Here I decided to remove "Why do YouTube views freeze at 301?" from "Numberphile" because its views count was artificialy fixed:
df_videos = df_videos[df_videos["video_title"] != "Why do YouTube views freeze at 301?"]
df_videos = df_videos.dropna()
df_videos = df_videos.reset_index(drop = True)

In [52]:
df_videos["comments_views_ratio"].describe()

count    109846.000000
mean          0.003138
std           0.006612
min           0.000000
25%           0.000647
50%           0.001555
75%           0.003415
max           0.440313
Name: comments_views_ratio, dtype: float64

In [53]:
df_videos.sort_values("comments_views_ratio").apply(lambda x: pd.concat([x.head(1), x.tail(1)]))

,channel_title,channel_id,video_title,video_id,video_upload_date,views,likes,dislikes,comments,age_days,likes_dislikes_ratio,comments_views_ratio
57561,"European Space Agency, ESA",UCIBaDdAbGlFDeS33shmlD0A,Paxi og månen vår: Faser og formørkelser,Q88PBr0Hj6U,2018-09-27 13:12:11+00:00,85249,128,22,0,1122.0,5.818182,0.000000
108733,UrAvgConsumer,UC9fSZHEh6XsRpX-xJc6lT3A,"Summer Giveaway Finale! (PS Vita, Nexus 7 2013...",T2LccukparY,2013-09-18 16:28:38+00:00,16218,2248,12,7141,2957.0,187.333333,0.440313


In [54]:
### Mean views per day

df_videos["mean_views_day"] = df_videos["views"]/df_videos["age_days"]
df_videos["mean_views_day"] = df_videos["mean_views_day"].replace([np.inf, -np.inf], np.nan)
df_videos = df_videos.dropna()
df_videos = df_videos.reset_index(drop = True)

In [55]:
df_videos["mean_views_day"].describe()

count    1.098460e+05
mean     9.710691e+02
std      8.986223e+03
min      7.012623e-04
25%      9.232827e+00
50%      5.030444e+01
75%      2.946639e+02
max      1.030206e+06
Name: mean_views_day, dtype: float64

In [56]:
df_videos.sort_values("mean_views_day").apply(lambda x: pd.concat([x.head(1), x.tail(1)]))

,channel_title,channel_id,video_title,video_id,video_upload_date,views,likes,dislikes,comments,age_days,likes_dislikes_ratio,comments_views_ratio,mean_views_day
23681,Business Insider,UCcyq283he07B7_KUX07mmtA,IGNITION 2017 LIVE - Day One: Morning Session,mppUHGYL_Bk,2017-11-27 14:58:54+00:00,1,31,16,0,1426.0,1.937500,0.000000,7.012623e-04
74143,Netflix,UCWOA1ZGywLbqmigxE4Qlvuw,RED NOTICE | Official Trailer | Netflix,Pj0wz7zu3Ms,2021-10-21 00:43:19+00:00,2060413,63908,1477,3385,2.0,43.268788,0.001643,1.030206e+06


In [58]:
# Save the dataframe:
df_videos.to_csv("data/videos_data.csv",
                 sep = ";",
                 index = False)